In [1]:
import sys
!{sys.executable} -m pip install pysam --no-index
!{sys.executable} -m pip install intervaltree --no-index


Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


In [2]:
import pandas as pd
import numpy as np
import pysam
from scipy import stats
import glob
from intervaltree import Interval, IntervalTree

## 1. Help functions

In [3]:
HAPPY_COLUMNS = ['Type', 'Filter', 'TRUTH.TP', 'QUERY.FP', 'TRUTH.FN', 'METRIC.Recall', 'METRIC.Precision', 'METRIC.F1_Score']

def load_targets_bed(filename):
    targets = dict()
    with open(filename, 'rt') as ifile:
        for line in ifile:
            chrom, start, stop = line.rstrip().split()
            targets.setdefault(chrom, IntervalTree()).addi(int(start) + 1, int(stop) + 1) # because BED is 0-based and IntervalTree stores half-open intervals.
    return targets

def read_af(filename):
    with pysam.VariantFile(filename, 'r') as ifile:
        for record in ifile:
            if 'ASJ_AF' not in record.info or 'ASJ_AN' not in record.info:
                continue
            if 'TOMPED_AF' not in record.info:
                continue
            if len(record.alts) > 1: # skip multi-allelic
                continue
            if record.info['ASJ_AN'] < 1000: # keep only those with sufficient ASJ sample size in gnomad 
                continue
            asj_af = record.info.get('ASJ_AF', [0.0])[0]
            if (asj_af <= 0) or (asj_af >= 1.0): # remove those which are monomorphic in ASJ according to gnomad
                continue
            topmed_af = record.info.get('TOMPED_AF', [0.0])[0]
            if (topmed_af <= 0) or (topmed_af >= 1.0): # remove those which are monomorphic in TOPMed
                continue
            if record.samples['QUERY']['BD'] == 'UNK' or record.samples['TRUTH']['BD'] == 'UNK': # variants outside the truth set regions
                continue
            if record.samples['TRUTH']['BD'] == 'N': # skip halfcalls such as "./1"
                continue
            targets = WEGS_TARGETS[record.chrom]
            entry = {
                     'Name': f'{record.chrom}:{record.pos}:{record.ref}:{record.alts[0]}',
                     'ASJ_AF': asj_af,
                     'TOPMED_AF': topmed_af,
                     'FOLD_CHANGE_AF': asj_af / topmed_af,
                     'IN_TARGET': targets.overlaps(record.start - 1) or targets.overlaps(record.stop - 1),
                     'AC_LESS': False,
                     'AC_GREATER': False,
                     'AC_SAME': False
                    }
            if record.samples['TRUTH']['BD'] == '.':
                truth_gt = 0
            else:
                truth_gt = sum(record.samples['TRUTH']['GT'])
            if record.samples['QUERY']['BD'] == '.':
                query_gt = 0
            else:
                query_gt = sum(record.samples['QUERY']['GT'])            
            if truth_gt > query_gt:
                entry['AC_LESS'] = True
            elif truth_gt < query_gt:
                entry['AC_GREATER'] = True
            else:
                entry['AC_SAME'] = True
            yield(entry)
            
def read_bad_variant_calls(filename):
    with pysam.VariantFile(filename, 'r') as ifile:
        for record in ifile:
            if 'ASJ_AF' not in record.info or 'ASJ_AN' not in record.info:
                continue
            if 'TOMPED_AF' not in record.info:
                continue
            if len(record.alts) > 1: # skip multi-allelic
                continue
            if record.info['ASJ_AN'] < 1000: # keep only those with sufficient ASJ sample size in gnomad 
                continue
            asj_af = record.info.get('ASJ_AF', [0.0])[0]
            if (asj_af <= 0) or (asj_af >= 1.0): # remove those which are monomorphic in ASJ according to gnomad
                continue
            topmed_af = record.info.get('TOMPED_AF', [0.0])[0]
            if (topmed_af <= 0) or (topmed_af >= 1.0): # remove those which are monomorphic in TOPMed
                continue
            if record.samples['QUERY']['BD'] == 'UNK' or record.samples['TRUTH']['BD'] == 'UNK': # variants outside the truth set regions
                continue
            if record.samples['TRUTH']['BD'] == 'N': # skip halfcalls such as "./1"
                continue
            if record.samples['TRUTH']['BD'] == '.':
                truth_gt = 0
            else:
                truth_gt = sum(record.samples['TRUTH']['GT'])
            if record.samples['QUERY']['BD'] == '.':
                query_gt = 0
            else:
                query_gt = sum(record.samples['QUERY']['GT'])            
            if truth_gt != query_gt:
                yield f'{record.chrom}:{record.pos}:{record.ref}:{record.alts[0]}'
                
def read_tp_snvs(filename):
    with pysam.VariantFile(filename, 'r') as ifile:
        for record in ifile.fetch():
            if len(record.alts) > 1: # skip multi-allelic
                continue
            if len(record.ref) != 1 or len(record.alts[0]) != 1: # keep only SNVs
                continue
            if record.samples['QUERY']['BD'] == 'UNK' or record.samples['TRUTH']['BD'] == 'UNK': # variants outside the truth set regions
                continue
            if record.samples['TRUTH']['BD'] == 'N': # skip halfcalls such as "./1"
                continue
            targets = WEGS_TARGETS[record.chrom]
            if not (targets.overlaps(record.start - 1) or targets.overlaps(record.stop - 1)): # use only variants within target regions
                continue
            if record.samples['TRUTH']['BD'] == '.':
                truth_gt = 0
            else:
                truth_gt = sum(record.samples['TRUTH']['GT'])
            if record.samples['QUERY']['BD'] == '.':
                query_gt = 0
            else:
                query_gt = sum(record.samples['QUERY']['GT'])            
            if truth_gt != query_gt: # skip not TP
                continue
            yield f'{record.chrom}:{record.pos}:{record.ref}:{record.alts[0]}'


## 2. Input files

In [4]:
SAMPLE_ANNOTATIONS_FILE = 'sample_annotations.txt'
WEGS_TARGETS_BED_FILE = 'WES_targets/SureSelectHumanAllExonV7.b37.Target.bed'
TOPMED_ARRAY_BENCHMARK_DIR = 'TOPMed_array_imputation_analysis'
WEGS_BENCHMARK_DIR = 'Variant_calling_analysis/WEGS/'
TP_ONLY_IN_WEGS_DIR = 'TOPMed_array_imputation_analysis/Only_in_WEGS'


## 3. Load data

In [5]:
# Load sample annotations
df_samples = pd.read_csv(SAMPLE_ANNOTATIONS_FILE, header = 0, sep = '\t')

hgid2naid = {
    'HG002': 'NA24385',
    'HG003': 'NA24149',
    'HG004': 'NA24143'
}

hgid2rel = {
    'HG002': 'Son',
    'HG003': 'Father',
    'HG004': 'Mother'
}

WEGS_TARGETS = load_targets_bed(WEGS_TARGETS_BED_FILE)

### 3.1 Load TOPMed imputed variants from array

In [6]:
df_array_imputed_variants = []
for row_index, row in df_samples[['HGID', 'Family']].drop_duplicates().iterrows():
    filepath = f'{TOPMED_ARRAY_BENCHMARK_DIR}/Genomewide/{row.HGID}_{hgid2naid[row.HGID]}_{row.Family}.dose.b37.happy_benchmark.summary.csv'
    df = pd.read_csv(filepath, usecols = HAPPY_COLUMNS).rename(columns = {
        'METRIC.Recall': 'Recall', 
        'METRIC.Precision': 'Precision',
        'METRIC.F1_Score': 'F1_Score',
        'TRUTH.TP': 'TP',
        'QUERY.FP': 'FP',
        'TRUTH.FN': 'FN'
    })
    df['Total_called'] = df['TP'] + df['FP']
    df['HGID'] = row.HGID
    df_array_imputed_variants.append(df)
df_array_imputed_variants = pd.concat(df_array_imputed_variants).reset_index(drop = True)

df_array_imputed_target_variants = []
for row_index, row in df_samples[['HGID', 'Family']].drop_duplicates().iterrows():
    filepath = f'{TOPMED_ARRAY_BENCHMARK_DIR}/Targets/{row.HGID}_{hgid2naid[row.HGID]}_{row.Family}.dose.b37.happy_benchmark.summary.csv'
    df = pd.read_csv(filepath, usecols = HAPPY_COLUMNS).rename(columns = {
        'METRIC.Recall': 'Recall', 
        'METRIC.Precision': 'Precision',
        'METRIC.F1_Score': 'F1_Score',
        'TRUTH.TP': 'TP',
        'QUERY.FP': 'FP',
        'TRUTH.FN': 'FN'
    })
    df['Total_called'] = df['TP'] + df['FP']
    df['HGID'] = row.HGID
    df_array_imputed_target_variants.append(df)
df_array_imputed_target_variants = pd.concat(df_array_imputed_target_variants).reset_index(drop = True)

### 3.2 Load WEGS variant calls

In [7]:
df_wegs_variants = []
for row_index, row in df_samples.iterrows():
    if row.Plexing == 1:
        continue
    for wgs_depth in [2, 5]:
        filepath = f'{WEGS_BENCHMARK_DIR}/{row.Plexing}plex_{wgs_depth}X_genomewide/{row.Filename}.norm.hard_filter.happy_benchmark.summary.csv'
        df = pd.read_csv(filepath, usecols = HAPPY_COLUMNS).rename(columns = {
            'METRIC.Recall': 'Recall', 
            'METRIC.Precision': 'Precision',
            'METRIC.F1_Score': 'F1_Score',
            'TRUTH.TP': 'TP',
            'QUERY.FP': 'FP',
            'TRUTH.FN': 'FN'
        })
        df['Total_called'] = df['TP'] + df['FP']
        df['ID'] = row.Filename
        df['HGID'] = row.HGID
        df['Batch'] = row.Batch
        df['Plexing'] = row.Plexing
        df['WGS_depth'] = wgs_depth
        df_wegs_variants.append(df)
df_wegs_variants = pd.concat(df_wegs_variants).reset_index(drop = True)

df_wegs_target_variants = []
for row_index, row in df_samples.iterrows():
    if row.Plexing == 1:
        continue
    for wgs_depth in [2, 5]:
        filepath = f'{WEGS_BENCHMARK_DIR}/{row.Plexing}plex_{wgs_depth}X/{row.Filename}.norm.hard_filter.happy_benchmark.summary.csv'
        df = pd.read_csv(filepath, usecols = HAPPY_COLUMNS).rename(columns = {
            'METRIC.Recall': 'Recall', 
            'METRIC.Precision': 'Precision',
            'METRIC.F1_Score': 'F1_Score',
            'TRUTH.TP': 'TP',
            'QUERY.FP': 'FP',
            'TRUTH.FN': 'FN'
        })
        df['Total_called'] = df['TP'] + df['FP']
        df['ID'] = row.Filename
        df['HGID'] = row.HGID
        df['Batch'] = row.Batch
        df['Plexing'] = row.Plexing
        df['WGS_depth'] = wgs_depth
        df_wegs_target_variants.append(df)
df_wegs_target_variants = pd.concat(df_wegs_target_variants).reset_index(drop = True)


### 3.3 Load variant calls annotated with allele frequencies from TOPMed and gnomAD

In [8]:
imputed_arrays = {}
for hgid, naid in hgid2naid.items():
    filename = f'{TOPMED_ARRAY_BENCHMARK_DIR}/Genomewide_with_AF/{hgid}_{naid}_{hgid2rel[hgid]}.dose.b37.happy_benchmark.vcf.gz'
    imputed_arrays[hgid] = pd.DataFrame(read_af(filename))
    
wegs_bad_4p2x = dict()
for hgid, naid in hgid2naid.items():
    wegs_bad_4p2x[hgid] = []
    for filename in glob.glob(f'{WEGS_BENCHMARK_DIR}/4plex_2X_genomewide_with_AF/{hgid}_{naid}_*.norm.hard_filter.happy_benchmark.vcf.gz'):
        wegs_bad_4p2x[hgid].append(set(read_bad_variant_calls(filename)))

wegs_bad_8p5x = dict()
for hgid, naid in hgid2naid.items():
    wegs_bad_8p5x[hgid] = []
    for filename in glob.glob(f'{WEGS_BENCHMARK_DIR}/8plex_5X_genomewide_with_AF/{hgid}_{naid}_*.norm.hard_filter.happy_benchmark.vcf.gz'):
        wegs_bad_8p5x[hgid].append(set(read_bad_variant_calls(filename)))

[W::bcf_hdr_register_hrec] The definition of Flag "INFO/IMPORT_FAIL" is invalid, forcing Number=0
[E::idx_find_and_load] Could not retrieve index file for 'TOPMed_array_imputation_analysis/Genomewide_with_AF/HG002_NA24385_Son.dose.b37.happy_benchmark.vcf.gz'
[W::bcf_hdr_register_hrec] The definition of Flag "INFO/IMPORT_FAIL" is invalid, forcing Number=0
[E::idx_find_and_load] Could not retrieve index file for 'TOPMed_array_imputation_analysis/Genomewide_with_AF/HG003_NA24149_Father.dose.b37.happy_benchmark.vcf.gz'
[W::bcf_hdr_register_hrec] The definition of Flag "INFO/IMPORT_FAIL" is invalid, forcing Number=0
[E::idx_find_and_load] Could not retrieve index file for 'TOPMed_array_imputation_analysis/Genomewide_with_AF/HG004_NA24143_Mother.dose.b37.happy_benchmark.vcf.gz'
[W::bcf_hdr_register_hrec] The definition of Flag "INFO/IMPORT_FAIL" is invalid, forcing Number=0
[E::idx_find_and_load] Could not retrieve index file for 'Variant_calling_analysis/WEGS//4plex_2X_genomewide_with_AF/HG

## 4. Figures and tables

### TABLE: Precision and recall rates of variants imputed using the TOPMed reference panel inside WES target regions (PART 1 - SNVs)

In [9]:
data = {
    'Sample': [],
    'Imputed TP': [],
    'Imputed FN': [],
    'Imputed FP': [],
    'Imputed Precision': [],
    'Imputed Recall': [],
    'WEGS 4P2X TP': [],
    'WEGS 4P2X FN': [],
    'WEGS 4P2X FP': [],
    'WEGS 4P2X Precision': [],
    'WEGS 4P2X Recall': [],
    'WEGS 8P5X TP': [],
    'WEGS 8P5X FN': [],
    'WEGS 8P5X FP': [],
    'WEGS 8P5X Precision': [],
    'WEGS 8P5X Recall': [],
}

for hgid, naid in hgid2naid.items():
    data['Sample'].append(hgid)
    
    df_temp = df_array_imputed_target_variants[df_array_imputed_target_variants.HGID == hgid]
    
    df_temp = df_temp[(df_temp.Type == 'SNP') & (df_temp.Filter == 'ALL')][['TP', 'FN', 'FP', 'Recall', 'Precision']]
    
    data['Imputed TP'].append(f'{df_temp.TP.values[0]:,}')
    data['Imputed FP'].append(f'{df_temp.FP.values[0]:,}')
    data['Imputed FN'].append(f'{df_temp.FN.values[0]:,}')
    data['Imputed Precision'].append(f'{df_temp.Precision.values[0]:.4f}')
    data['Imputed Recall'].append(f'{df_temp.Recall.values[0]:.4f}')

    df_temp = df_wegs_target_variants[df_wegs_target_variants.HGID == hgid]
    df_temp = df_temp[(df_temp.Type == 'SNP') & (df_temp.Filter == 'ALL') & (df_temp.Plexing == 4) & (df_temp.WGS_depth == 2)]
    data['WEGS 4P2X TP'].append(f'{np.mean(df_temp.TP):,.0f} ({stats.sem(df_temp.TP):,.0f})')
    data['WEGS 4P2X FP'].append(f'{np.mean(df_temp.FP):,.0f} ({stats.sem(df_temp.FP):,.0f})')
    data['WEGS 4P2X FN'].append(f'{np.mean(df_temp.FN):,.0f} ({stats.sem(df_temp.FN):,.0f})')
    data['WEGS 4P2X Precision'].append(f'{np.mean(df_temp.Precision):.4f} ({stats.sem(df_temp.Precision):,.4f})')
    data['WEGS 4P2X Recall'].append(f'{np.mean(df_temp.Recall):.4f} ({stats.sem(df_temp.Recall):,.4f})')
    
    df_temp = df_wegs_target_variants[df_wegs_target_variants.HGID == hgid]
    df_temp = df_temp[(df_temp.Type == 'SNP') & (df_temp.Filter == 'ALL') & (df_temp.Plexing == 8) & (df_temp.WGS_depth == 5)]
    data['WEGS 8P5X TP'].append(f'{np.mean(df_temp.TP):,.0f} ({stats.sem(df_temp.TP):,.0f})')
    data['WEGS 8P5X FP'].append(f'{np.mean(df_temp.FP):,.0f} ({stats.sem(df_temp.FP):,.0f})')
    data['WEGS 8P5X FN'].append(f'{np.mean(df_temp.FN):,.0f} ({stats.sem(df_temp.FN):,.0f})')
    data['WEGS 8P5X Precision'].append(f'{np.mean(df_temp.Precision):.4f} ({stats.sem(df_temp.Precision):,.4f})')
    data['WEGS 8P5X Recall'].append(f'{np.mean(df_temp.Recall):.4f} ({stats.sem(df_temp.Recall):,.4f})')
    
pd.DataFrame(data)

,Sample,Imputed TP,Imputed FN,Imputed FP,Imputed Precision,Imputed Recall,WEGS 4P2X TP,WEGS 4P2X FN,WEGS 4P2X FP,WEGS 4P2X Precision,WEGS 4P2X Recall,WEGS 8P5X TP,WEGS 8P5X FN,WEGS 8P5X FP,WEGS 8P5X Precision,WEGS 8P5X Recall
0,HG002,"21,458","1,285",165,0.9924,0.9435,"22,379 (7)",364 (7),446 (6),0.9805 (0.0002),0.9840 (0.0003),"22,390 (5)",353 (5),449 (7),0.9803 (0.0003),0.9845 (0.0002)
1,HG003,"21,477","1,112",111,0.9949,0.9508,"22,231 (9)",358 (9),425 (4),0.9813 (0.0001),0.9841 (0.0004),"22,249 (6)",340 (6),406 (5),0.9821 (0.0002),0.9850 (0.0003)
2,HG004,"21,228","1,315",181,0.9915,0.9417,"22,195 (9)",348 (9),406 (7),0.9820 (0.0003),0.9846 (0.0004),"22,197 (5)",346 (5),402 (3),0.9822 (0.0001),0.9847 (0.0002)


### TABLE: Precision and recall rates of variants imputed using the TOPMed reference panel inside WES target regions (PART 2 - InDels)

In [10]:
data = {
    'Sample': [],
    'Imputed TP': [],
    'Imputed FN': [],
    'Imputed FP': [],
    'Imputed Precision': [],
    'Imputed Recall': [],
    'WEGS 4P2X TP': [],
    'WEGS 4P2X FN': [],
    'WEGS 4P2X FP': [],
    'WEGS 4P2X Precision': [],
    'WEGS 4P2X Recall': [],
    'WEGS 8P5X TP': [],
    'WEGS 8P5X FN': [],
    'WEGS 8P5X FP': [],
    'WEGS 8P5X Precision': [],
    'WEGS 8P5X Recall': [],
}

for hgid, naid in hgid2naid.items():
    data['Sample'].append(hgid)
    
    df_temp = df_array_imputed_target_variants[df_array_imputed_target_variants.HGID == hgid]
    
    df_temp = df_temp[(df_temp.Type == 'INDEL') & (df_temp.Filter == 'ALL')][['TP', 'FN', 'FP', 'Recall', 'Precision']]
    
    data['Imputed TP'].append(f'{df_temp.TP.values[0]:,}')
    data['Imputed FP'].append(f'{df_temp.FP.values[0]:,}')
    data['Imputed FN'].append(f'{df_temp.FN.values[0]:,}')
    data['Imputed Precision'].append(f'{df_temp.Precision.values[0]:.4f}')
    data['Imputed Recall'].append(f'{df_temp.Recall.values[0]:.4f}')

    df_temp = df_wegs_target_variants[df_wegs_target_variants.HGID == hgid]
    df_temp = df_temp[(df_temp.Type == 'INDEL') & (df_temp.Filter == 'ALL') & (df_temp.Plexing == 4) & (df_temp.WGS_depth == 2)]
    data['WEGS 4P2X TP'].append(f'{np.mean(df_temp.TP):,.0f} ({stats.sem(df_temp.TP):,.0f})')
    data['WEGS 4P2X FP'].append(f'{np.mean(df_temp.FP):,.0f} ({stats.sem(df_temp.FP):,.0f})')
    data['WEGS 4P2X FN'].append(f'{np.mean(df_temp.FN):,.0f} ({stats.sem(df_temp.FN):,.0f})')
    data['WEGS 4P2X Precision'].append(f'{np.mean(df_temp.Precision):.4f} ({stats.sem(df_temp.Precision):,.4f})')
    data['WEGS 4P2X Recall'].append(f'{np.mean(df_temp.Recall):.4f} ({stats.sem(df_temp.Recall):,.4f})')
    
    df_temp = df_wegs_target_variants[df_wegs_target_variants.HGID == hgid]
    df_temp = df_temp[(df_temp.Type == 'INDEL') & (df_temp.Filter == 'ALL') & (df_temp.Plexing == 8) & (df_temp.WGS_depth == 5)]
    data['WEGS 8P5X TP'].append(f'{np.mean(df_temp.TP):,.0f} ({stats.sem(df_temp.TP):,.0f})')
    data['WEGS 8P5X FP'].append(f'{np.mean(df_temp.FP):,.0f} ({stats.sem(df_temp.FP):,.0f})')
    data['WEGS 8P5X FN'].append(f'{np.mean(df_temp.FN):,.0f} ({stats.sem(df_temp.FN):,.0f})')
    data['WEGS 8P5X Precision'].append(f'{np.mean(df_temp.Precision):.4f} ({stats.sem(df_temp.Precision):,.4f})')
    data['WEGS 8P5X Recall'].append(f'{np.mean(df_temp.Recall):.4f} ({stats.sem(df_temp.Recall):,.4f})')
    
pd.DataFrame(data)

,Sample,Imputed TP,Imputed FN,Imputed FP,Imputed Precision,Imputed Recall,WEGS 4P2X TP,WEGS 4P2X FN,WEGS 4P2X FP,WEGS 4P2X Precision,WEGS 4P2X Recall,WEGS 8P5X TP,WEGS 8P5X FN,WEGS 8P5X FP,WEGS 8P5X Precision,WEGS 8P5X Recall
0,HG002,271,411,8,0.9713,0.3974,648 (3),34 (3),57 (1),0.9197 (0.0017),0.9498 (0.0050),647 (2),35 (2),58 (3),0.9177 (0.0038),0.9481 (0.0028)
1,HG003,257,433,8,0.9698,0.3725,649 (2),41 (2),52 (2),0.9254 (0.0019),0.9409 (0.0027),651 (1),39 (1),52 (1),0.9265 (0.0018),0.9432 (0.0019)
2,HG004,259,384,7,0.9738,0.4028,616 (2),28 (2),43 (2),0.9357 (0.0035),0.9572 (0.0026),615 (2),28 (2),40 (2),0.9393 (0.0033),0.9568 (0.0031)


### TABLE: Precision and recall rates of variants imputed using the TOPMed reference panel genome-wide (PART 1 - SNVs)

In [11]:
data = {
    'Sample': [],
    'Imputed TP': [],
    'Imputed FN': [],
    'Imputed FP': [],
    'Imputed Precision': [],
    'Imputed Recall': [],
    'WEGS 4P2X TP': [],
    'WEGS 4P2X FN': [],
    'WEGS 4P2X FP': [],
    'WEGS 4P2X Precision': [],
    'WEGS 4P2X Recall': [],
    'WEGS 8P5X TP': [],
    'WEGS 8P5X FN': [],
    'WEGS 8P5X FP': [],
    'WEGS 8P5X Precision': [],
    'WEGS 8P5X Recall': [],
}

for hgid, naid in hgid2naid.items():
    data['Sample'].append(hgid)
    
    df_temp = df_array_imputed_variants[df_array_imputed_variants.HGID == hgid]
    
    df_temp = df_temp[(df_temp.Type == 'SNP') & (df_temp.Filter == 'ALL')][['TP', 'FN', 'FP', 'Recall', 'Precision']]
    
    data['Imputed TP'].append(f'{df_temp.TP.values[0]:,}')
    data['Imputed FP'].append(f'{df_temp.FP.values[0]:,}')
    data['Imputed FN'].append(f'{df_temp.FN.values[0]:,}')
    data['Imputed Precision'].append(f'{df_temp.Precision.values[0]:.4f}')
    data['Imputed Recall'].append(f'{df_temp.Recall.values[0]:.4f}')

    df_temp = df_wegs_variants[df_wegs_variants.HGID == hgid]
    df_temp = df_temp[(df_temp.Type == 'SNP') & (df_temp.Filter == 'ALL') & (df_temp.Plexing == 4) & (df_temp.WGS_depth == 2)]
    data['WEGS 4P2X TP'].append(f'{np.mean(df_temp.TP):,.0f} ({stats.sem(df_temp.TP):,.0f})')
    data['WEGS 4P2X FP'].append(f'{np.mean(df_temp.FP):,.0f} ({stats.sem(df_temp.FP):,.0f})')
    data['WEGS 4P2X FN'].append(f'{np.mean(df_temp.FN):,.0f} ({stats.sem(df_temp.FN):,.0f})')
    data['WEGS 4P2X Precision'].append(f'{np.mean(df_temp.Precision):.4f} ({stats.sem(df_temp.Precision):,.4f})')
    data['WEGS 4P2X Recall'].append(f'{np.mean(df_temp.Recall):.4f} ({stats.sem(df_temp.Recall):,.4f})')
    
    df_temp = df_wegs_variants[df_wegs_variants.HGID == hgid]
    df_temp = df_temp[(df_temp.Type == 'SNP') & (df_temp.Filter == 'ALL') & (df_temp.Plexing == 8) & (df_temp.WGS_depth == 5)]
    data['WEGS 8P5X TP'].append(f'{np.mean(df_temp.TP):,.0f} ({stats.sem(df_temp.TP):,.0f})')
    data['WEGS 8P5X FP'].append(f'{np.mean(df_temp.FP):,.0f} ({stats.sem(df_temp.FP):,.0f})')
    data['WEGS 8P5X FN'].append(f'{np.mean(df_temp.FN):,.0f} ({stats.sem(df_temp.FN):,.0f})')
    data['WEGS 8P5X Precision'].append(f'{np.mean(df_temp.Precision):.4f} ({stats.sem(df_temp.Precision):,.4f})')
    data['WEGS 8P5X Recall'].append(f'{np.mean(df_temp.Recall):.4f} ({stats.sem(df_temp.Recall):,.4f})')
    
pd.DataFrame(data)

,Sample,Imputed TP,Imputed FN,Imputed FP,Imputed Precision,Imputed Recall,WEGS 4P2X TP,WEGS 4P2X FN,WEGS 4P2X FP,WEGS 4P2X Precision,WEGS 4P2X Recall,WEGS 8P5X TP,WEGS 8P5X FN,WEGS 8P5X FP,WEGS 8P5X Precision,WEGS 8P5X Recall
0,HG002,"3,212,631","140,048","21,239",0.9934,0.9582,"1,323,307 (13,949)","2,029,372 (13,949)","293,787 (1,558)",0.8183 (0.0023),0.3947 (0.0042),"1,873,500 (1,770)","1,479,179 (1,770)","242,833 (353)",0.8853 (0.0002),0.5588 (0.0005)
1,HG003,"3,181,715","132,428","17,060",0.9947,0.9600,"1,518,733 (9,194)","1,795,410 (9,194)","325,169 (886)",0.8236 (0.0013),0.4583 (0.0028),"2,151,757 (2,243)","1,162,386 (2,243)","228,413 (270)",0.9040 (0.0002),0.6493 (0.0007)
2,HG004,"3,202,593","137,878","20,523",0.9936,0.9587,"1,159,478 (6,227)","2,180,992 (6,227)","291,399 (438)",0.7991 (0.0011),0.3471 (0.0019),"1,671,569 (3,941)","1,668,902 (3,941)","253,243 (540)",0.8684 (0.0005),0.5004 (0.0012)


### TABLE: Precision and recall rates of variants imputed using the TOPMed reference panel genome-wide (PART 2 - InDels)

In [12]:
data = {
    'Sample': [],
    'Imputed TP': [],
    'Imputed FN': [],
    'Imputed FP': [],
    'Imputed Precision': [],
    'Imputed Recall': [],
    'WEGS 4P2X TP': [],
    'WEGS 4P2X FN': [],
    'WEGS 4P2X FP': [],
    'WEGS 4P2X Precision': [],
    'WEGS 4P2X Recall': [],
    'WEGS 8P5X TP': [],
    'WEGS 8P5X FN': [],
    'WEGS 8P5X FP': [],
    'WEGS 8P5X Precision': [],
    'WEGS 8P5X Recall': [],
}

for hgid, naid in hgid2naid.items():
    data['Sample'].append(hgid)
    
    df_temp = df_array_imputed_variants[df_array_imputed_variants.HGID == hgid]
    
    df_temp = df_temp[(df_temp.Type == 'INDEL') & (df_temp.Filter == 'ALL')][['TP', 'FN', 'FP', 'Recall', 'Precision']]
    
    data['Imputed TP'].append(f'{df_temp.TP.values[0]:,}')
    data['Imputed FP'].append(f'{df_temp.FP.values[0]:,}')
    data['Imputed FN'].append(f'{df_temp.FN.values[0]:,}')
    data['Imputed Precision'].append(f'{df_temp.Precision.values[0]:.4f}')
    data['Imputed Recall'].append(f'{df_temp.Recall.values[0]:.4f}')

    df_temp = df_wegs_variants[df_wegs_variants.HGID == hgid]
    df_temp = df_temp[(df_temp.Type == 'INDEL') & (df_temp.Filter == 'ALL') & (df_temp.Plexing == 4) & (df_temp.WGS_depth == 2)]
    data['WEGS 4P2X TP'].append(f'{np.mean(df_temp.TP):,.0f} ({stats.sem(df_temp.TP):,.0f})')
    data['WEGS 4P2X FP'].append(f'{np.mean(df_temp.FP):,.0f} ({stats.sem(df_temp.FP):,.0f})')
    data['WEGS 4P2X FN'].append(f'{np.mean(df_temp.FN):,.0f} ({stats.sem(df_temp.FN):,.0f})')
    data['WEGS 4P2X Precision'].append(f'{np.mean(df_temp.Precision):.4f} ({stats.sem(df_temp.Precision):,.4f})')
    data['WEGS 4P2X Recall'].append(f'{np.mean(df_temp.Recall):.4f} ({stats.sem(df_temp.Recall):,.4f})')
    
    df_temp = df_wegs_variants[df_wegs_variants.HGID == hgid]
    df_temp = df_temp[(df_temp.Type == 'INDEL') & (df_temp.Filter == 'ALL') & (df_temp.Plexing == 8) & (df_temp.WGS_depth == 5)]
    data['WEGS 8P5X TP'].append(f'{np.mean(df_temp.TP):,.0f} ({stats.sem(df_temp.TP):,.0f})')
    data['WEGS 8P5X FP'].append(f'{np.mean(df_temp.FP):,.0f} ({stats.sem(df_temp.FP):,.0f})')
    data['WEGS 8P5X FN'].append(f'{np.mean(df_temp.FN):,.0f} ({stats.sem(df_temp.FN):,.0f})')
    data['WEGS 8P5X Precision'].append(f'{np.mean(df_temp.Precision):.4f} ({stats.sem(df_temp.Precision):,.4f})')
    data['WEGS 8P5X Recall'].append(f'{np.mean(df_temp.Recall):.4f} ({stats.sem(df_temp.Recall):,.4f})')
    
pd.DataFrame(data)

,Sample,Imputed TP,Imputed FN,Imputed FP,Imputed Precision,Imputed Recall,WEGS 4P2X TP,WEGS 4P2X FN,WEGS 4P2X FP,WEGS 4P2X Precision,WEGS 4P2X Recall,WEGS 8P5X TP,WEGS 8P5X FN,WEGS 8P5X FP,WEGS 8P5X Precision,WEGS 8P5X Recall
0,HG002,"199,347","323,042","2,089",0.9896,0.3816,"132,493 (1,957)","389,896 (1,957)","55,634 (632)",0.7043 (0.0008),0.2536 (0.0037),"198,767 (351)","323,622 (351)","68,172 (123)",0.7447 (0.0003),0.3805 (0.0007)
1,HG003,"195,885","304,908","1,795",0.9909,0.3911,"154,695 (1,172)","346,098 (1,172)","63,485 (423)",0.7091 (0.0003),0.3089 (0.0023),"232,882 (368)","267,911 (368)","72,598 (142)",0.7624 (0.0002),0.4650 (0.0007)
2,HG004,"197,298","310,976","2,006",0.9899,0.3882,"111,566 (788)","396,708 (788)","50,204 (374)",0.6897 (0.0004),0.2195 (0.0015),"168,158 (651)","340,116 (651)","62,636 (264)",0.7287 (0.0002),0.3308 (0.0013)


### TABLE: Imputed variants, their allele frequencies, and overlap with true positive (TP) variants in WEGS outside WES target regions.

In [13]:
categories = ['AC_SAME', 'AC_LESS', 'AC_GREATER']

data = {
    'Sample': [],
    'Truth vs imputed': [],
    'N variants': [],
    'WEGS 4P2X': [],
    'WEGS 8P5X': [],
    'AF ASJ': [],
    'AF TOPMed': [],
    'AF fold-change': [],
}

for hgid, df in imputed_arrays.items():
    for cat in categories:
        data['Sample'].append(hgid)
        
        df_temp = df[(df.IN_TARGET != True) & (df[cat] == True)]
        
        wegs = [len(df_temp[~df_temp.Name.isin(names)]) / len(df_temp) * 100 for names in wegs_bad_4p2x[hgid]]
        data['WEGS 4P2X'].append(f'{np.mean(wegs):,.2f} ({stats.sem(wegs):,.2f})')
        
        wegs = [len(df_temp[~df_temp.Name.isin(names)]) / len(df_temp) * 100 for names in wegs_bad_8p5x[hgid]]
        data['WEGS 8P5X'].append(f'{np.mean(wegs):,.2f} ({stats.sem(wegs):,.2f})')

        data['Truth vs imputed'].append(cat)
        data['N variants'].append(f'{len(df_temp):,}')

        median = np.median(df_temp.ASJ_AF)
        q25, q75 = np.quantile(df_temp.ASJ_AF, [0.25, 0.75])
        data['AF ASJ'].append(f'{median:.3f} ({q25:.3f}-{q75:.3f})')

        median = np.median(df_temp.TOPMED_AF)
        q25, q75 = np.quantile(df_temp.TOPMED_AF, [0.25, 0.75])
        data['AF TOPMed'].append(f'{median:.3f} ({q25:.3f}-{q75:.3f})')

        median = np.median(df_temp.FOLD_CHANGE_AF)
        q25, q75 = np.quantile(df_temp.FOLD_CHANGE_AF, [0.25, 0.75])
        data['AF fold-change'].append(f'{median:.3f} ({q25:.3f}-{q75:.3f})')
        
pd.DataFrame(data)


,Sample,Truth vs imputed,N variants,WEGS 4P2X,WEGS 8P5X,AF ASJ,AF TOPMed,AF fold-change
0,HG002,AC_SAME,"3,258,732",38.44 (0.42),55.16 (0.05),0.460 (0.239-0.703),0.442 (0.231-0.680),1.042 (0.925-1.197)
1,HG002,AC_LESS,"15,233",41.85 (0.48),59.31 (0.11),0.234 (0.059-0.483),0.215 (0.043-0.453),1.136 (0.926-1.509)
2,HG002,AC_GREATER,"17,161",90.55 (0.05),92.75 (0.02),0.329 (0.147-0.536),0.340 (0.157-0.539),1.007 (0.840-1.209)
3,HG003,AC_SAME,"3,225,810",44.84 (0.28),64.45 (0.07),0.461 (0.239-0.704),0.443 (0.231-0.681),1.043 (0.927-1.198)
4,HG003,AC_LESS,"12,665",46.75 (0.34),67.09 (0.08),0.222 (0.051-0.496),0.215 (0.033-0.458),1.129 (0.930-1.537)
5,HG003,AC_GREATER,"13,855",92.38 (0.04),94.60 (0.03),0.302 (0.134-0.477),0.318 (0.145-0.501),1.006 (0.822-1.210)
6,HG004,AC_SAME,"3,247,624",33.65 (0.19),49.22 (0.12),0.457 (0.237-0.702),0.440 (0.230-0.679),1.041 (0.923-1.196)
7,HG004,AC_LESS,"15,357",38.89 (0.30),55.00 (0.12),0.219 (0.056-0.465),0.205 (0.042-0.440),1.114 (0.915-1.565)
8,HG004,AC_GREATER,"16,457",89.38 (0.05),91.94 (0.02),0.325 (0.142-0.534),0.330 (0.153-0.539),1.022 (0.841-1.239)


### EXTRA TABLE: Imputed variants, their allele frequencies, and overlap with true positive (TP) variants in WEGS in targets only.

In [14]:
data = {
    'Sample': [],
    'Truth vs imputed': [],
    'N variants 4P+2X': [],
    '% variants 4P+2X': [],
    'N variants 8P+5X': [],
    '% variants 8P+5X': [],
}

categories = ['AC_SAME', 'AC_LESS', 'AC_GREATER']

for hgid, df in imputed_arrays.items():
    for cat in categories:
        data['Sample'].append(hgid)
        if cat == 'All':
            df_temp = df[(df.IN_TARGET == True)]
        else:
            df_temp = df[(df.IN_TARGET == True) & (df[cat] == True)]
            
        data['Truth vs imputed'].append(cat)
            
        n = []
        perc = []
        for wegs_bad in wegs_bad_4p2x[hgid]:
            df_temp_subset = df_temp[(~df_temp.Name.isin(wegs_bad))]
            n.append(len(df_temp_subset))
            perc.append(len(df_temp_subset) / len(df_temp) * 100)
        data['N variants 4P+2X'].append(f'{np.mean(n):.1f}')
        data['% variants 4P+2X'].append(f'{np.mean(perc):.1f}')
        
        n = []
        perc = []
        for wegs_bad in wegs_bad_4p2x[hgid]:
            df_temp_subset = df_temp[(~df_temp.Name.isin(wegs_bad))]
            n.append(len(df_temp_subset))
            perc.append(len(df_temp_subset) / len(df_temp) * 100)
        data['N variants 8P+5X'].append(f'{np.mean(n):.1f}')
        data['% variants 8P+5X'].append(f'{np.mean(perc):.1f}')
        
        
pd.DataFrame(data)

,Sample,Truth vs imputed,N variants 4P+2X,% variants 4P+2X,N variants 8P+5X,% variants 8P+5X
0,HG002,AC_SAME,20387.2,99.6,20387.2,99.6
1,HG002,AC_LESS,117.8,98.9,117.8,98.9
2,HG002,AC_GREATER,134.0,99.3,134.0,99.3
3,HG003,AC_SAME,20263.2,99.6,20263.2,99.6
4,HG003,AC_LESS,82.0,100.0,82.0,100.0
5,HG003,AC_GREATER,83.0,100.0,83.0,100.0
6,HG004,AC_SAME,20169.2,99.7,20169.2,99.7
7,HG004,AC_LESS,183.0,98.9,183.0,98.9
8,HG004,AC_GREATER,119.2,96.2,119.2,96.2


### EXTRA TABLE: Target region SNVs missed by imputation but capruted by WEGS

In [15]:
### THE INPUT FILES FOR THE TABLE WERE GENERATED USING THIS CODE:

# ### 1. Load True Positives in imputation, WEGS 4P2X, and WEGS 8P5X

# imputed_arrays_target_tp = {}
# for hgid, naid in hgid2naid.items():
#     filename = f'{TOPMED_ARRAY_BENCHMARK_DIR}/Genomewide_with_AF/{hgid}_{naid}_{hgid2rel[hgid]}.dose.b37.happy_benchmark.vcf.gz'
#     imputed_arrays_target_tp[hgid] = pd.DataFrame(read_tp_snvs(filename))
    
# wegs_target_tp_4p2x = dict()
# for hgid, naid in hgid2naid.items():
#     wegs_target_tp_4p2x[hgid] = []
#     for filename in glob.glob(f'{WEGS_BENCHMARK_DIR}/4plex_2X_genomewide_with_AF/{hgid}_{naid}_*.norm.hard_filter.happy_benchmark.vcf.gz'):
#         wegs_target_tp_4p2x[hgid].append(set(read_tp_snvs(filename)))

# wegs_target_tp_8p5x = dict()
# for hgid, naid in hgid2naid.items():
#     wegs_target_tp_8p5x[hgid] = []
#     for filename in glob.glob(f'{WEGS_BENCHMARK_DIR}/8plex_5X_genomewide_with_AF/{hgid}_{naid}_*.norm.hard_filter.happy_benchmark.vcf.gz'):
#         wegs_target_tp_8p5x[hgid].append(set(read_tp_snvs(filename)))
        
# ### 2. Extract unique variants (which were True Positives only in WEGS)
# for hgid, naid in hgid2naid.items():
#     array_variants = imputed_arrays_target_tp[hgid]
#     for i, wegs_variants in enumerate(wegs_target_tp_4p2x[hgid], 1):
#         unique_to_array = array_variants.difference(wegs_variants)
#         unique_to_wegs = wegs_variants.difference(array_variants)
#         print(hgid, len(array_variants), len(wegs_variants), len(unique_to_array), len(unique_to_wegs))
#         with open(f'{hgid}.{i}.4P2X.unique_to_wegs.txt', 'wt') as ofile:
#             for name in sorted(unique_to_wegs):
#                 chrom, pos, ref, alt = name.split(':')
#                 ofile.write(f'{chrom} {pos} . {ref} {alt} . . .' + '\n')
                
# for hgid, naid in hgid2naid.items():
#     array_variants = imputed_arrays_target_tp[hgid]
#     for i, wegs_variants in enumerate(wegs_target_tp_8p5x[hgid], 1):
#         unique_to_array = array_variants.difference(wegs_variants)
#         unique_to_wegs = wegs_variants.difference(array_variants)
#         print(hgid, len(array_variants), len(wegs_variants), len(unique_to_array), len(unique_to_wegs))
#         with open(f'{hgid}.{i}.8P5X.unique_to_wegs.txt', 'wt') as ofile:
#             for name in sorted(unique_to_wegs):
#                 chrom, pos, ref, alt = name.split(':')
#                 ofile.write(f'{chrom} {pos} . {ref} {alt} . . .' + '\n')

# ### 3. RUN ONLINE VEP (with display most severe consequence option) on the *.unique_to_wegs.txt files, and save output as *.unique_to_wegs.vep.txt



In [16]:
data = []
for hgid, naid in hgid2naid.items():
    for wegs_name in ['4P2X', '8P5X']:
        for filename in glob.glob(f'{TP_ONLY_IN_WEGS_DIR}/{hgid}.*.{wegs_name}.unique_to_wegs.vep.txt'):
            df_temp = pd.read_table(filename)
            total = len(df_temp)
            n_nonsynonymous = len(df_temp[df_temp.Consequence.isin({'missense_variant', 'stop_gained', 'stop_lost', 'start_lost'})])
            n_splice = len(df_temp[df_temp.Consequence.isin({'splice_acceptor_variant', 'splice_donor_variant'})])
            data.append({
                'Sequencing': f'WEGS_{wegs_name}',
                'Sample': hgid,
                'N_only_in_WEGS': total,
                'N_nonsynonymous_only_in_WEGS': n_nonsynonymous,
                'N_splice_only_in_WEGS': n_splice,
            })

pd.DataFrame(data)

,Sequencing,Sample,N_only_in_WEGS,N_nonsynonymous_only_in_WEGS,N_splice_only_in_WEGS
0,WEGS_4P2X,HG002,976,569,3
1,WEGS_4P2X,HG002,970,559,3
2,WEGS_4P2X,HG002,977,565,3
3,WEGS_4P2X,HG002,977,565,3
4,WEGS_8P5X,HG002,981,569,3
5,WEGS_8P5X,HG002,967,558,3
6,WEGS_8P5X,HG002,986,571,3
7,WEGS_8P5X,HG002,970,558,3
8,WEGS_8P5X,HG002,981,567,3
9,WEGS_4P2X,HG003,834,485,2
